# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import os
import sys
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
import pickle

nltk.download(['punkt', 'punkt_tab', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
sys.path.append(os.path.abspath(".."))

import src.config as conf

In [3]:
# load data from database
engine = create_engine('sqlite:///' + "../" + conf.DATABASE_FILENAME)
df = pd.read_sql_table(conf.TABLE_NAME, engine)
X = df['message']
Y = df.iloc[:, 4:]
category_names = list(df.columns[4:])

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [6]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def my_tokenizer(text):
    '''
    Clean and tokenize the text in input

    Args:
        text (str): input text

    Returns:
        clean_tokens (list): tokens obtained from the input text
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
# Pipiline simple version
simple_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = my_tokenizer, token_pattern = None))
    , ('tfidf', TfidfTransformer())
    , ('clf', MultiOutputClassifier(RandomForestClassifier()))])

simple_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__monotonic_cst', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [10]:
%%time
simple_pipeline.fit(X_train, Y_train)

CPU times: total: 12min 52s
Wall time: 12min 55s


Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function my_tokenizer at 0x000001DB43AE5260>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
Y_pred = simple_pipeline.predict(X_test)
    
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print('Category {}: {} '.format(i, category_names[i]))
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i], zero_division = 0))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])))

Category 0: related 
              precision    recall  f1-score   support

           0       0.76      0.27      0.40      1555
           1       0.81      0.97      0.88      4964
           2       0.29      0.14      0.19        35

    accuracy                           0.80      6554
   macro avg       0.62      0.46      0.49      6554
weighted avg       0.79      0.80      0.76      6554

Accuracy 0.8001220628623741


Category 1: request 
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5458
           1       0.88      0.43      0.58      1096

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

Accuracy 0.8945682026243515


Category 2: offer 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6518
           1       0.00      0.00      0.00        36

    acc

Let's print it in a more readable way:

In [12]:
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), zero_division = 0))#, target_names = category_names))

              precision    recall  f1-score   support

           0       0.88      0.43      0.58      1096
           1       0.00      0.00      0.00        36
           2       0.79      0.59      0.68      2711
           3       0.73      0.04      0.08       527
           4       0.70      0.07      0.13       352
           5       0.60      0.02      0.03       171
           6       0.25      0.01      0.02       122
           7       0.73      0.04      0.07       207
           8       0.00      0.00      0.00         0
           9       0.97      0.20      0.34       409
          10       0.88      0.37      0.52       754
          11       0.90      0.19      0.32       581
          12       0.71      0.05      0.10        95
          13       0.83      0.03      0.06       165
          14       1.00      0.01      0.02        87
          15       0.43      0.01      0.03       224
          16       0.77      0.12      0.21       288
          17       0.59    

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
%%time
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2))
    , 'vect__max_df': (0.5, 0.75, 1.0)
    , 'tfidf__use_idf': (True, False)
    #, 'clf__estimator__n_estimators': [50, 100, 200]
    , 'clf__estimator__min_samples_split': [2, 3, 4]
}

simple_pipeline_cv = GridSearchCV(simple_pipeline, param_grid = parameters)

CPU times: total: 0 ns
Wall time: 0 ns


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
%%time
simple_pipeline_cv.fit(X_train, Y_train)
print('Best Parameters:', simple_pipeline_cv.best_params_)

In [ ]:
Y_pred = simple_pipeline_cv.predict(X_test)
    
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), zero_division = 0, target_names = category_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    '''
    Extracts whether a text starts with a verb or 'RT' as a custom feature.
    
    This transformer uses NLTK to tokenize and POS-tag sentences in the input text.
    It checks if the first word of any sentence is a verb (VB or VBP) or 'RT'.
    '''
    def starting_verb(self, text):
        '''
        Check if text starts with a verb or 'RT'.
        
        Args:
            text (str): Input text to analyze.
            
        Returns:
            bool: True if any sentence starts with a verb (VB, VBP) or 'RT', False otherwise.
        '''
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(my_tokenizer(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        '''
        Fit the transformer (no-op).
        
        Args:
            x: Input data, ignored.
            y: Target values, ignored (default=None).
            
        Returns:
            self: Returns the instance itself.
        '''
        return self

    def transform(self, X):
        '''
        Transform input texts by extracting starting verb feature.
        
        Args:
            X: Input data, array-like of strings.
            
        Returns:
            pd.DataFrame: DataFrame with boolean values indicating if each text
                         starts with a verb or 'RT'.
        '''
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer = my_tokenizer, token_pattern = None))
            , ('tfidf', TfidfTransformer())]))
        , ('starting_verb', StartingVerbExtractor())]))
    , ('clf', RandomForestClassifier())])

pipeline.get_params()

In [ ]:
%%time
pipeline.fit(X_train, Y_train)

In [ ]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), zero_division = 0, target_names = category_names)) 

In [ ]:
%%time
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

pipeline_cv = GridSearchCV(pipeline, param_grid = parameters)

In [ ]:
%%time
pipeline_cv.fit(X_train, Y_train)
print('Best Parameters:', pipeline_cv.best_params_)

In [ ]:
Y_pred = pipeline_cv.predict(X_test)
    
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), zero_division = 0, target_names = category_names))

### 9. Export your model as a pickle file

In [14]:
model = simple_pipeline
pickle.dump(model, open("../" + conf.MODEL_PICKLE_FILENAME, 'wb'))

In [15]:
# Load the model from pickle file

model = pickle.load(open("../" + conf.MODEL_PICKLE_FILENAME, 'rb'))

category_predicted = model.predict(['Storm at sacred heart of jesus'])[0]
print(category_predicted)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0]


In [16]:
def get_df_from_database(database_filename = conf.DATABASE_FILENAME):
    '''
    Return dataframe from the database

    Args:
        database_filename (str): database filename. Default value DATABASE_FILENAME

    Returns:
        df (pandas.DataFrame): dataframe containing the data 
    '''
    engine = create_engine('sqlite:///' + database_filename)
    return pd.read_sql_table(conf.TABLE_NAME, engine)

In [17]:
def get_category_names(database_filename = conf.DATABASE_FILENAME):
    '''
    Return category names

    Args:
        database_filename (str): database filename. Default value DATABASE_FILENAME

    Returns:
        category_names (list): list of the category names
    '''
    df = get_df_from_database(database_filename)
    return list(df.columns[4:])

print(get_category_names("../" + conf.DATABASE_FILENAME))

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [18]:
def get_predicted_category_names(category_predicted):
    '''
    Return the names of the categories corresponding to predicted one in input

    Args:
        category_predicted (list): list of predicted category

    Returns:
        predicted_category_names (list): list of predicted category names
    '''
    category_names = get_category_names("../" + conf.DATABASE_FILENAME)
    return [category_names[i] for i in range(len(category_predicted)) if category_predicted[i] == 1]

print(get_predicted_category_names(category_predicted))

['related', 'weather_related', 'storm']


In [19]:
def get_genre_distribution(database_filename = conf.DATABASE_FILENAME):
    '''
    Return message genre distribution

    Args:
        database_filename (str): database filename. Default value DATABASE_FILENAME

    Returns:
        genre_distribution (dict): dictionary of message genre distribution (genre, count)
    '''
    df = get_df_from_database(database_filename)
    return df.groupby('genre').count()['message'].to_dict()

print(get_genre_distribution("../" + conf.DATABASE_FILENAME))

{'direct': 10766, 'news': 13054, 'social': 2396}


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.